In [1]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
from imageio import imread

In [2]:
df = pd.read_csv("data/df.csv")

In [3]:
n = 5000

In [24]:
all_q = []
all_des = []
all_title = []
all_index = []

for i in range(n):
    if("(Novel)" in df.loc[i, "title"] or "(Light Novel)" in df.loc[i, "title"] or "(Webcomic)" in df.loc[i, "title"] or "(Doujinshi)" in df.loc[i, "title"]):
        continue
    if(df.loc[i, "q"]):
        all_q.append(df.loc[i, "q"].split('\n')[0])
        all_des.append(str(df.loc[i, "description"]))
        all_title.append(str(df.loc[i, "title"]))
        all_index.append(i)

In [5]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

/Users/madness/Desktop/manga-recommendation/env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
model.save("model/models")

In [6]:
embeddings_des = model.encode(all_des, convert_to_tensor=True)

In [7]:
torch.save(embeddings_des, "data/embeddings_des.pt")

In [8]:
embeddings_des = torch.load("data/embeddings_des.pt")

In [9]:
embeddings_des

tensor([[-0.0764, -0.0286, -0.0090,  ...,  0.0280, -0.0281,  0.0467],
        [-0.0447,  0.0627, -0.0609,  ..., -0.0362,  0.0145, -0.0067],
        [ 0.0194,  0.1176, -0.0242,  ..., -0.0544,  0.0220,  0.0003],
        ...,
        [ 0.0158,  0.0444, -0.0221,  ..., -0.0364,  0.0448,  0.0304],
        [-0.0974, -0.0387, -0.0111,  ...,  0.0098, -0.0145,  0.0147],
        [-0.0500, -0.0670,  0.0436,  ..., -0.0264, -0.0468, -0.0197]])

In [32]:
val_q = all_q

In [33]:
all_val_q = []
label = []
for i in range(len(val_q)):
    embedding_q = model.encode(val_q[i], convert_to_tensor=True)
    idxx = all_q.index(val_q[i])
    label.append(idxx)
    all_val_q.append(embedding_q)

In [56]:
query = input("Enter query :")

In [57]:
query_en = model.encode(query, convert_to_tensor=True)

In [58]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
cosine_scores = util.cos_sim(query_en, embeddings_des)

all_idx = torch.topk(cosine_scores.flatten(), 5).indices
print(all_idx)
for i in all_idx:
    print("name :", df.loc[all_index[int(i)], "title"])
    img_url = df.loc[int(i), "cover"]
    # img = imread(img_url)
    # plt.imshow(img)
    # plt.show()

tensor([3154, 3719, 2632, 3987,  312])
name : Border
name : The New Kindaichi Files
name : Tantei Xeno to Nanatsu no Satsujin Misshitsu
name : Kechonpa
name : Brutal: Satsujin Keisatsukan no Kokuhaku


In [61]:
acc = 0
n_val = len(all_val_q)
topk = 5

for i in range(n_val):
    cosine_scores = util.cos_sim(all_val_q[i], embeddings_des)
    #print(len(cosine_scores[0]))
    #max_val = max(cosine_scores[0])
    all_idx = torch.topk(cosine_scores.flatten(), topk).indices
    ans = []
    for i in all_idx:
        ans.append(all_index[int(i)])

    #check idex in all_des==idx
    if(all_index[label[i]] in ans):
        acc += 1

In [62]:
print("acc =", acc/len(all_val_q))

acc = 0.9818353385232366
